In [1]:
import FinanceDataReader as fdr

In [2]:
stock_list = fdr.StockListing('KRX').dropna() # 상장폐지 종목은 삭제

In [3]:
stock_list = stock_list.head(3) # 시간 단축을 위하여 3개만 활용

In [4]:
stock_list

,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
0,060310,KOSDAQ,3S,전자부품 제조업,반도체 웨이퍼 캐리어,2002-04-23,03월,김세완,http://www.3sref.com,서울특별시
1,095570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",2015-08-21,12월,"박대현, 손삼달",http://www.ajnet.co.kr,서울특별시
2,006840,KOSPI,AK홀딩스,기타 금융업,지주사업,1999-08-11,12월,"채형석, 이석주(각자 대표이사)",http://www.aekyunggroup.co.kr,서울특별시


In [5]:
import time
import pandas as pd

In [6]:
daily = pd.DataFrame()
for code, name in stock_list[['Symbol', 'Name']].values:
    print(code, name)
    date = '2022-05-02'
    ohlcv = fdr.DataReader(code, date, date) # 하루치 데이터 불러오기
    ohlcv['Code'] = code
    ohlcv['Name'] = name
    daily = pd.concat([daily, ohlcv])

060310 3S
095570 AJ네트웍스
006840 AK홀딩스


[Pandas 기초] 데이터프레임 합치기(merge, join, concat)
- https://yganalyst.github.io/data_handling/Pd_12/

Pandas(판다스) - FinanceDataReader을 사용해 주식 데이터를 dataframe으로 불러오기 
- https://blog.naver.com/PostView.naver?blogId=ji0eeeee&logNo=221527166066&parentCategoryNo=&categoryNo=11&viewDate=&isShowPopularPosts=true&from=search

열 추가, 값 수정, 데이터 합치기
- https://nalara12200.tistory.com/162 

[python] 팬더스 데이터 프레임의 인덱스를 열로 변환하는 방법은 무엇입니까?
- http://daplus.net/python-%ED%8C%AC%EB%8D%94-%EB%8D%B0%EC%9D%B4%ED%84%B0-%ED%94%84%EB%A0%88%EC%9E%84%EC%9D%98-%EC%9D%B8%EB%8D%B1%EC%8A%A

In [7]:
daily

,Open,High,Low,Close,Volume,Change,Code,Name
Date,,,,,,,,
2022-05-02,3200,3275,3200,3250,69707,0.003086,060310,3S
2022-05-02,6630,6660,6420,6650,30618,0.000000,095570,AJ네트웍스
2022-05-02,21550,21650,21050,21350,14068,-0.006977,006840,AK홀딩스


In [10]:
import pandas as pd
import numpy as np
import time, json, requests
from datetime import datetime, timedelta, date

import pymysql # python에서 mysql을 사용하는 패키지

import sqlalchemy # sql 접근 및 관리를 도와주는 패키지
from sqlalchemy import create_engine

In [11]:
server = '127.0.0.1'
user = 'root'
passwd = 'abcd123456'
db = 'whsf1_db'

# sqlalchemy의 create_engine을 이용하여 DB 연결
engine = create_engine('mysql+pymysql://{}:{}@{}/{}?charset=utf8'.format(user, passwd, server, db))


In [12]:
daily.to_sql(name='kospi_adjusted1',con=engine,if_exists='append',index=False,
             dtype = { # sql에 저장할 때, 데이터 유형도 설정할 수 있다.

                 'Open' : sqlalchemy.types.BIGINT(),
                 'High' : sqlalchemy.types.BIGINT(),
                 'Low' : sqlalchemy.types.BIGINT(),
                 'Close' : sqlalchemy.types.BIGINT(),
                 'Volume' : sqlalchemy.types.BIGINT(),
                 'Change' : sqlalchemy.types.FLOAT(),
                 'Code' : sqlalchemy.types.VARCHAR(10),
                 'Name' : sqlalchemy.types.TEXT(),
                 'Date' : sqlalchemy.types.DATE(),
             }
            )

In [16]:
# DB 데이터 저장 확인
sql = 'select * from kospi_adjusted1'
db_data = pd.read_sql(sql, engine)
db_data

,Open,High,Low,Close,Volume,Change,Code,Name
0,3200,3275,3200,3250,69707,0.003086,060310,3S
1,6630,6660,6420,6650,30618,0.000000,095570,AJ네트웍스
2,21550,21650,21050,21350,14068,-0.006977,006840,AK홀딩스


In [17]:
# DB 접속 해제
engine.dispose()
#cur.close()
#conn.close()